# Exploratory Data Analysis

In the following, I analyze the dataset underlying the ML model for scheduling standby drivers.
<br>
I want to clarify which variables are interrelated, how they are distributed and thus find out which ones are suitable for the model as features.

#### Fragen?
* Are the data seasonal?
* What is the connection between how many standby drivers are needed?
* Frage 3

In [79]:
import warnings
warnings.simplefilter(action='ignore')

In [80]:
from datetime import datetime, date 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [ ]:
import platform
print(platform.python_version())

### Load and inspect the data:

In [ ]:
df = pd.read_csv('/Users/kata/Desktop/GitHub/standby_duty_forecasting/sickness_table.csv') #, index_col=0)
df['date'] = pd.to_datetime(df['date'])

Ideally, the data is already in chronological order and the time stamps are equidistant in time series. This is already the case in our data: The time interval is one day and the data is already in chronological order. Therefore, we do not have to do this additional data preparation step.
<br>
This column is provided in string format. Let's convert it to the datetime64[ns] data type.

[Quelle](https://www.kaggle.com/code/iamleonie/intro-to-time-series-forecasting/notebook)

In [ ]:
df.head().style.set_properties(subset=['date'], **{'background-color': 'dodgerblue'})

* **date:** entry date
* **n_sick:** number of drivers called sick on duty
* **calls:** number of emergency calls
* **n_duty:** number of drivers on duty available
* **n_sby:** number of standby resources available
* **sby_need:** number of standbys, which are activated on a given day
* **dafted:** number of additional drivers needed due to not enough standbys

In [ ]:
# Check time intervals
df = df.sort_values(by='date')
df['Time_Interval'] = df.date - df.date.shift(1)

df[['date', 'Time_Interval']].head()

The data should be in chronological order and the timestamps should be equidistant in time series. 
<br>
&rarr; IT IS

Let's look at numerical summaries:

In [ ]:
df.describe(include = 'all')

At least 4074 calls were received during the period, maximum 11.850. On average, 7919.53 calls were received.
<br>
On average, 34.7 drivers are needed on standby. However, since the maximum is 555, it is worth looking for outliers here.

Check for NaN-values:

In [ ]:
df[df.isnull().any(axis=1)]

The following data types exist:

In [ ]:
df.info()

So sehen die Were in verschiedenen Spalten aus:

In [ ]:
df.n_duty.unique()

This means, there is only three options of drivers on duty. The values increase over time.

In [ ]:
df.n_sby.unique()

There are always 90 drivers on standby.

In [ ]:
def min_max(data):
    print(data.min())
    print(data.max())

In [ ]:
min_max(df.date)

The data covers a period of three years, from April 1st 2016 to May 27th 2019.

Correlation matrix between all the features we are examining and our y-variable:

In [ ]:
corr = df.corr()

print(corr)

### Investigate graphically

Make pairplots to investigate data further and spot correlations:

In [ ]:
sns.set_theme(style="white")

sns.pairplot(df)

n_sick und calls sind saisonal, bei sby_need und calls sieht man, dass es erst ab 8000 calls Standby-Fahrer braucht.

The distribution of "sby_need" looks interesting, this is how the histogram looks in detail:

In [ ]:
plt.figure(figsize=(12, 8))
plt.clf()
plt.hist(df.sby_need, bins=50)
plt.title('Standby drivers needed', fontsize=14)
plt.xlabel('Number of standby drivers needed')
plt.show()

In [ ]:
new_df2 = df.groupby(['sby_need']).size()
print(new_df2)

Standby drivers were not needed in nearly three-quarters of the cases. However, there are also individual cases in which several hundred were needed.
<br>
Another graphic will illustrate this.

In [ ]:
# needed???
plt.clf()
plt.figure(figsize=(12, 8))
plt.plot(new_df2)
plt.axis([0, 555, 0, 5])
plt.title('Required standby drivers', fontsize=14)
plt.show()

Most of the time no standby drivers were needed, otherwise between 2 and 5. It could become problematic that there are numerous individual cases where hundreds were needed.

Visualize data over the years:

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(df.date, df.sby_need)
#plt.xticks([204, 650, 900], ['19.4.2016', '16.5.2016', '4.6.2016'])
plt.show()

### Further Correlations

We observe that the number of drivers increases over the years. Is there a correlation with the number of calls?

In [ ]:
plt.figure(figsize=(14,10))
sns.set_style("whitegrid")
sns.lineplot(x='date', y='calls', hue='n_duty', data=df, linewidth = 0.8)
#plt.xticks([0, 275, 640, 1151], ['01.04.2016', '01.01.2017', '01.01.2018', '27.05.2019'])

plt.xticks(rotation=90)
plt.legend(labels=["1700","1800", "1900"], title = "Drivers on Duty")
plt.xlabel('Date', fontsize=13)
plt.ylabel('Calls', fontsize=13)
plt.title('Development of drivers on duty and emergency calls', fontsize=18)

plt.savefig("dev_n_duty_calls.svg")
plt.show()

The graph shows that the number of drivers on duty increases with the number of calls.

In [ ]:
print(pearsonr(df.calls, df.n_duty)[0])

According to Pearson's correlation coefficient r, the two variables are slightly positively correlated with a value of 0.36.

What about the number of calls and the number of drivers on sick leave?

In [ ]:
plt.figure(figsize=(14,10))
sns.set_style("whitegrid")
sns.regplot(x='calls', y='n_sick', data=df)
plt.xticks(rotation=90)

plt.title('Development of sick drivers and emergency calls', fontsize=18)
plt.xlabel('Calls', fontsize=13)
plt.ylabel('Sick drivers', fontsize=13)

plt.show()

If the number of calls increases, the number of sick drivers slightly increases

In [ ]:
print(pearsonr(df.calls, df.n_sick)[0])

The variables are weakly positively correlated.

The pairplot above suggests a correlation between calls and drivers needed. Pearson's r confirms this:

In [ ]:
print(pearsonr(df.calls, df.sby_need)[0])

The variables "n_sick" and "sby_need" are less related than thought:

In [ ]:
print(pearsonr(df.n_sick, df.sby_need)[0])

### check if data is stationary

Some time-series models, such as such as ARIMA, assume that the underlying data is stationary. Stationarity describes that the time-series has
<br>
* constant mean and mean is not time-dependent
* constant variance and variance is not time-dependent
* constant covariance and covariance is not time-dependent


If a time series has a specific (stationary) behavior over a given time interval, then it can be assumed that the time series will behave the same at a later time.
<br>
Time series with trend and/or seasonality are not stationary. Trend indicates that the mean is not constant over time and seasonality indicates that the variance is not constant over time.

[Source](https://www.kaggle.com/code/iamleonie/intro-to-time-series-forecasting/notebook)

Visual check:

In [ ]:
#sby_need
rolling_window = 52
sns.lineplot(x=df.date, y=df.sby_need, color='indianred')
sns.lineplot(x=df.date, y=df.sby_need.rolling(rolling_window).mean(), color='black', label='rolling mean')
sns.lineplot(x=df.date, y=df.sby_need.rolling(rolling_window).std(), color='blue', label='rolling std')


Mean and variance is not constant.
<br>
&rarr; doesn't seem to be stationary


In [ ]:
# calls
sns.lineplot(x=df.date, y=df.calls, color='indianred')
sns.lineplot(x=df.date, y=df.calls.rolling(rolling_window).mean(), color='black', label='rolling mean')
sns.lineplot(x=df.date, y=df.calls.rolling(rolling_window).std(), color='blue', label='rolling std')


Mean and variance is not constant.
<br>
&rarr; doesn't seem to be stationary


Summary statistics:

In [ ]:
def mean_var(data):
    X = data.values
    split = round(len(X) / 2)
    X1, X2 = X[0:split], X[split:]
    mean1, mean2 = X1.mean(), X2.mean()
    var1, var2 = X1.var(), X2.var()
    print('mean1=%f, mean2=%f' % (mean1, mean2))
    print('variance1=%f, variance2=%f' % (var1, var2))

check for standby drivers:

In [ ]:
mean_var(df.sby_need)

check for calls:

In [ ]:
mean_var(df.calls)

Mean and variance values for both are different and not in the same ball park.

Augmented Dickey-Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
#sby_need
result = adfuller(df.sby_need.values)
adf_stat = result[0]
p_val = result[1]
crit_val_1 = result[4]['1%']
crit_val_5 = result[4]['5%']
crit_val_10 = result[4]['10%']

In [ ]:
def visualize_adfuller_results(series, title, ax):
    result = adfuller(series)
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']

    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'forestgreen' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'gold'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'orange'
    else:
        linecolor = 'indianred'
    sns.lineplot(x=df.date, y=series, ax=ax, color=linecolor)
    ax.set_title(f'ADF Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    ax.set_ylabel(ylabel=title, fontsize=14)

In [ ]:

f, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 7))
visualize_adfuller_results(df.sby_need.values, 'sby_need', ax[0])
visualize_adfuller_results(df.calls.values, 'calls', ax[1])

plt.tight_layout()
plt.show()

sby_need:
<br>
**p-value <= significance level (0.05)**: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.
<br>
(**ADF statistic < critical value**: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.)

calls:
<br>
**p-value > significance level**: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary. 
<br>
(**ADF statistic < critical value**: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.)

&rarr; calls not stationary

Running the test prints the test statistic value of -6. The more negative this statistic, the more likely we are to reject the null hypothesis (we have a stationary dataset).
<br>
As part of the output, we get a look-up table to help determine the ADF statistic. We can see that our statistic value of -6 is less than the value of -3.436 at 1%.
<br>
This suggests that we can reject the null hypothesis with a significance level of less than 1% (i.e. a low probability that the result is a statistical fluke).
<br>
We can also look at the p-value, which is 0. This means we can easily reject the null and consider the distribution as stationary.
<br>
Rejecting the null hypothesis means that the process has no unit root, and in turn that the time series is stationary or does not have time-dependent structure.
<br>
<br>
&rarr; the data is  stationary ([Tutorial](https://machinelearningmastery.com/time-series-data-stationary-python/)
)

### KPSS test for stationarity

<br>
KPSS test is a statistical test to check for stationarity of a series around a deterministic trend. Like ADF test, the KPSS test is also commonly used to analyse the stationarity of a series. However, it has couple of key differences compared to the ADF test in function and in practical usage. Therefore, is not safe to just use them interchangeably. We’ll discuss this detail with simplified examples.

[Resource](https://www.machinelearningplus.com/time-series/kpss-test-for-stationarity/)

A key difference from ADF test is the null hypothesis of the KPSS test is that the series is stationary.
<br>
So practically, the interpretaion of p-value is just the opposite to each other.

In [ ]:
from statsmodels.tsa.stattools import kpss

In [ ]:
# eher den nehmen!!!!!!!!!

def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)


print('CALLS')
kpss_test(df.calls)
print('SBY NEED')
kpss_test(df.sby_need)

there is evidence for rejecting the null hypothesis in favor of the alternative. Hence, both series are non-stationary as per the KPSS test.

In [ ]:

# with or without regression?
# zweite quelle: https://www.machinelearningplus.com/time-series/kpss-test-for-stationarity/

In [ ]:
def kpss_test2(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

print('CALLS')
kpss_test2(df.calls, regression="c")
print('SBY NEED')
kpss_test2(df.sby_need, regression="c")

FINAL RESULT: Calls non stationary
<br>
KPSS indicates non-stationarity and ADF indicates stationarity - The series is difference stationary. Differencing is to be used to make series stationary. The differenced series is checked for stationarity.

[Difference between KPSS and ADF](https://www.machinelearningplus.com/time-series/kpss-test-for-stationarity/)

# stationary calls???

In [ ]:
# make calls stationary???

## Transformation of non stationary calls

In [ ]:
# Log Transform of absolute values
# (Log transoform of negative values will return NaN)
df['calls_log'] = np.log(abs(df.calls))


In [ ]:
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 6))
visualize_adfuller_results(abs(df.calls), 'Absolute \n Calls', ax[0, 0])

sns.distplot(df.calls_log, ax=ax[0, 1])
visualize_adfuller_results(df.calls_log, 'Transformed \n Calls', ax[1, 0])

sns.distplot(df.calls_log, ax=ax[1, 1])

plt.tight_layout()
plt.show()

In [ ]:
#hä?? was hat das gebracht?

In [ ]:
# another way: differencing



In [ ]:
# First Order Differencing
calls_diff = np.diff(df.calls)
df['calls_diff_1'] = np.append([0], calls_diff)

In [ ]:
df.head()

In [ ]:
# The differencing can be reverted if the the first value before differencing is known. 
# In this case, we can accumulate all values with the function .cumsum() and add the first value of the original time series.


df.calls.equals(df.calls_diff_1.cumsum() + df.calls.iloc[0])


In [ ]:
print('p-value: ' + str(adfuller(df.calls_diff_1.dropna())[1]))

In [ ]:
# EDA fängt eig erst hier an

## seasonality

Check seasonality of the Standby drivers needed:

In [ ]:
ax = df.plot(x='date', y='sby_need', figsize=(12,6))
#plt.xticks([275, 640, 1005], ['01.01.2017', '01.01.2018', '01.01.2019'])
xcoords = [275, 640, 1005]
for xc in xcoords:
    plt.axvline(x=xc, color='black', linestyle='--')

seasonality of calls:

In [ ]:
ax = df.plot(x='date', y='calls', figsize=(12,6))
#plt.xticks([275, 640, 1005], ['01.01.2017', '01.01.2018', '01.01.2019'])
xcoords = [275, 640, 1005]
for xc in xcoords:
    plt.axvline(x=xc, color='black', linestyle='--')

seasonal decomposition:

In [ ]:
# split variable into components to see seasonal variation

def decompose(data):
    res = sm.tsa.seasonal_decompose(data, period=365, model='additive')
    fig = res.plot()
    fig.set_size_inches(14,7)
    plt.show()

In [ ]:
# decompose calls
decompose(df.calls)

In [ ]:
decompose(df.sby_need)

In [ ]:
# correlation matrix

# calculate autocorrelation

with this [resource](https://www.alpharithms.com/autocorrelation-time-series-python-432909/)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
# perform autocorrelation & visualization
def autocorr(data):
    res = sm.graphics.tsa.plot_acf(x=data, lags=100)
    res.set_size_inches(14,7)
    plt.show()

#The red shaded region is the confidence interval with a default value of α = 0.05. 
#Anything within this range represents a value that has no significant correlation with the most recent value.

In [ ]:
autocorr(df.sby_need)

In [ ]:
autocorr(df.calls)

In [ ]:
# get lags over confidence intervall
from statsmodels.tsa.stattools import acf

def get_lags(data):
    a, ci = acf(x=data, nlags=100, alpha=0.05, fft=True)
    centered_ci = ci - a[:,None]
    outside = np.abs(a) >= centered_ci[:,1]
    inside = ~outside

    return outside


In [ ]:
lags_sby = get_lags(df.sby_need)


In [ ]:
lags_calls = get_lags(df.calls)


In [ ]:
def lags_count(array):
    a = sum(x==True for x in array)
    return a

In [ ]:
lags_count(lags_sby)

In [ ]:
lags_count(lags_calls)

In [ ]:
# another way
# resource: https://www.kaggle.com/code/iamleonie/intro-to-time-series-forecasting/notebook

from pandas.plotting import autocorrelation_plot

In [ ]:
plt.figure(figsize=(14,8))
autocorrelation_plot(df.calls_diff_1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 8))

plot_acf(df.calls_diff_1,lags=100, ax=ax[0])
plot_pacf(df.calls_diff_1,lags=100, ax=ax[1])
plt.show()

# cross validation (WAS IS DAS)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit


In [ ]:
N_SPLITS = 3

X = df.date
y = df.calls

folds = TimeSeriesSplit(n_splits=N_SPLITS)

In [ ]:
f, ax = plt.subplots(nrows=N_SPLITS, ncols=2, figsize=(16, 9))

for i, (train_index, valid_index) in enumerate(folds.split(X)):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    sns.lineplot(x= X_train, y= y_train, ax=ax[i,0], color='dodgerblue', label='train')
    sns.lineplot(x= X_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
                 y= y_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
                 ax=ax[i,1], color='dodgerblue', label='train')

    for j in range(2):
        sns.lineplot(x= X_valid, y= y_valid, ax=ax[i, j], color='darkorange', label='validation')
    ax[i, 0].set_title(f"Rolling Window with Adjusting Training Size (Split {i+1})", fontsize=16)
    ax[i, 1].set_title(f"Rolling Window with Constant Training Size (Split {i+1})", fontsize=16)

for i in range(N_SPLITS):
    ax[i, 0].set_xlim([date(2016, 1, 1), date(2019, 6, 30)])
    ax[i, 1].set_xlim([date(2016, 1, 1), date(2019, 6, 30)])
plt.tight_layout()
plt.show()